In [231]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [232]:
cd /content/drive/MyDrive/TalkLikeMom/src/Classifier

/content/drive/MyDrive/PHD/TalkLikeMom/src/Classifier


In [233]:
import sys
sys.path.append('.')

In [234]:
# !pip install -q emoji pythainlp

In [235]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

from transformers import AutoTokenizer
import transformers
import torch
import numpy as np
import scipy as sp
# import shap

from utils import dump_jsonl, load_jsonl, set_random_seed
# import torch
# from transformers import AutoModelForSequenceClassification, AdamW, BertConfig

In [236]:
import pickle
def load_obj_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

In [7]:
# !pip install -q pythainlp emoji==1.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [237]:
import emoji
import pythainlp
from emoji import UNICODE_EMOJI

def is_emoji(s):
    for char in s:
        if char in UNICODE_EMOJI["en"]:
            return True
    return False

labelmap = {
    '1. Close': "1. Close",
    '2. Know each other': "2. Acquainted",
    "3. Don't know each other": "3. Unfamiliar",


    '0. Very respect': "0. Highly Respectful",
    '1. Respect': "1. Respectful",
    '2. Normal': "2. Normal",
    '3. Not respect': "3. Disrespectful"
}

def label2newlabel(label):
    if type(label) is list or type(label) is np.ndarray:
        return [labelmap[l] if l in labelmap else l for l in label ]

    if label in labelmap:
        return labelmap[label]
    return label




In [245]:
fn_abs = False
suffix = "abs" if fn_abs else ''

lexicon_paths = [
  f"Lexicons/v3/lexicon_task1_clse{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task2_clse{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task3_clse{suffix}.jsonl",

  f"Lexicons/v3/lexicon_task1_auth{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task2_auth{suffix}.jsonl",
  f"Lexicons/v3/lexicon_task3_auth{suffix}.jsonl"
]

def print_lexicon(lexicons, lexicons_by_feat):
    s = ""
    for label in sorted(lexicons.keys()):
        if label=="overall":
          continue

        for lextype in ["all", "pronoun", "particles", "misspelling"]:

          lexlabel = {
              "all": "All",
              "pronoun": "Pronoun",
              "particles": "Sentence-final Particle",
              "misspelling": "Spelling Variation",
          }




          if lextype=="all":
            s += "            "+label2newlabel(label) +f" & {lexlabel[lextype]} & "
            words = lexicons[label]
          else:
            s += "            " +f" & {lexlabel[lextype]} & "
            words = lexicons_by_feat[label][lextype]

          sorted_words = dict(sorted(words.items(), key=lambda item: -item[1]))
          for k in list(sorted_words.keys())[0:10]:
              unique = False

              if pythainlp.util.isthai(k):
                  s += "\\thaitext{"+k+"}, "
              elif is_emoji(k):
                  kk = ""
                  for ch in k:
                      if ch in UNICODE_EMOJI["en"]:
                        ch = emoji.demojize(ch)
                        ch = ch.replace("_", "-").replace(":", "")
                        ch = "\emoji{"+ch+"}"
                        kk += ch
                      else:
                        kk += ch

                  s += f"{kk}, "
              else:
                  s += f"{k}, "

          s += "\\\\\n"

        s += "            \hline\n"
    # s += "            \hline"
    # print(s)
    return s

texts = []
for path in lexicon_paths:
  lexicons, lexicons_by_feat = load_jsonl(path)
  s = print_lexicon(lexicons, lexicons_by_feat)
  texts.append(s)
  # print(s)
  # break

Loaded 2 records from Lexicons/v3/lexicon_task1_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task2_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task1_auth.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task2_auth.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_auth.jsonl


In [247]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (texts[0], texts[3]),
    (texts[1], texts[4]),
    (texts[2], texts[5]),
]

printed_text = ""
for section, (t1, t2) in zip(sections, table_contents):

    printed_text += "\subsection{"+section+"}"

    printed_text += '''
\\begin{longtable}[h]{
        p{\dimexpr 0.20\linewidth-2\\tabcolsep}
        p{\dimexpr 0.25\linewidth-2\\tabcolsep}
        p{\dimexpr 0.55\linewidth-2\\tabcolsep}
    }
        \hline
        Relationship Label  & Lexicon Category & Lexicons\\\\
        \hline
        \endfirsthead
        \hline
        Relationship Label  & Lexicon Category & Lexicons\\\\
        \hline
        \endhead


        \multicolumn{3}{l}{\\textit{Closeness}} \\\\
        \hline
'''
    printed_text += t1
    printed_text += '''
            \hline

        \multicolumn{3}{l}{\\textit{Respect}} \\\\
        \hline
'''

    printed_text += t2
    printed_text += '''
            \hline
\end{longtable}
\clearpage
\n\n
'''

# print(printed_text)

In [9]:
# lexicon_paths = [
#   # "Lexicons/v3/lexicon_task1_clse.jsonl",
#   "Lexicons/v3/lexicon_task2_clse.jsonl",
#   "Lexicons/v3/lexicon_task3_clse.jsonl",

#   # "Lexicons/v3/lexicon_task1_auth.jsonl",
#   # "Lexicons/v3/lexicon_task2_auth.jsonl",
#   # "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# allSelectedWords = set()
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)

#   selectedWords = set()
#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))

#     keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
#     keys = keys[0:200]
#     words = set(keys)
#     selectedWords.update(words)
#   print("Final", len(selectedWords))

#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))
#     for f in lexicons_by_feat[cat]:
#       feats = set(lexicons_by_feat[cat][f].keys())
#       # if f=="pronoun":
#       #   print(sorted(feats))
#       N = len(lexicons_by_feat[cat][f].keys())
#       print(f, len(feats&selectedWords), N, f"{len(feats&selectedWords)*100/N:.0f}%")

#   allSelectedWords.update(selectedWords)
#   print("==============")

# print(len(allSelectedWords))

In [10]:
# rows = []
# for word in allSelectedWords:
#   rows.append({
#       "คำ (Word)": word,
#       "สนิทกันมาก (Intimate)": "",
#       "สนิทกัน (Close)": "",
#       "แค่คนรู้จักกัน (Acquainted)": "",
#       "ไม่รู้จักกัน (Unfamiliar)": "",
#       "ไม่ชอบหน้ากัน  (Dislike)": "",
#       "บอกไม่ได้ (Cannot describe)": "",
#   })

# import pandas as pd
# df = pd.DataFrame(rows)
# df.to_csv("clse_lexicons.csv", index=False)

In [11]:
# lexicon_paths = [
#   # "Lexicons/v3/lexicon_task1_clse.jsonl",
#   # "Lexicons/v3/lexicon_task2_clse.jsonl",
#   # "Lexicons/v3/lexicon_task3_clse.jsonl",

#   # "Lexicons/v3/lexicon_task1_auth.jsonl",
#   "Lexicons/v3/lexicon_task2_auth.jsonl",
#   "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# allSelectedWords = set()
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)

#   selectedWords = set()
#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))

#     keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
#     keys = keys[0:200]
#     words = set(keys)
#     selectedWords.update(words)
#   print("Final", len(selectedWords))

#   for cat in lexicons_by_feat:
#     # print("all", len(lexicons[cat].keys()))
#     for f in lexicons_by_feat[cat]:
#       feats = set(lexicons_by_feat[cat][f].keys())
#       # if f=="pronoun":
#       #   print(sorted(feats))
#       N = len(lexicons_by_feat[cat][f].keys())
#       print(f, len(feats&selectedWords), N, f"{len(feats&selectedWords)*100/N:.0f}%")

#   allSelectedWords.update(selectedWords)
#   print("==============")

# print(len(allSelectedWords))

In [12]:
# rows = []
# for word in allSelectedWords:
#   rows.append({
#       "คำ (Word)": word,
#       "ให้เกียรติมาก (Highly Respectful)": "",
#       "ให้เกียรติ (Respectful)": "",
#       "ปกติ (Normal)": "",
#       "ไม่ให้เกียรติ (Disrespectful)": "",
#       "ไม่ให้เกียรติมาก (Highly Disrespectful)": "",
#       "บอกไม่ได้ (Cannot describe)": "",
#   })

# import pandas as pd
# df = pd.DataFrame(rows)
# df.to_csv("auth_lexicons.csv", index=False)

In [279]:
import pandas as pd
hum = pd.read_csv("closeness_u1.csv", skiprows=[0])
# hum = pd.read_csv("authority_u1.csv", skiprows=[0])

words = {}
columns = hum.columns
for idx, row in hum.iterrows():
  w = row["คำ (Word)"]


  for col in columns:
    if col in ["คำ (Word)", "Unnamed: 7", "บอกไม่ได้ \n(Cannot describe)"]:
      continue

    if pd.isna(row[col]):
      continue

    words[w] = col

In [249]:
# words

In [280]:
lexicon_paths = [
  # "Lexicons/v3/lexicon_task1_clse.jsonl",
  "Lexicons/v3/lexicon_task2_clse.jsonl",
  "Lexicons/v3/lexicon_task3_clse.jsonl",

  # "Lexicons/v3/lexicon_task1_auth.jsonl",
  # "Lexicons/v3/lexicon_task2_auth.jsonl",
  # "Lexicons/v3/lexicon_task3_auth.jsonl"
]

data = []


for path in lexicon_paths:
  lexicons, lexicons_by_feat = load_jsonl(path)
  for cat in lexicons:

    keys = [k for k, v in sorted(lexicons[cat].items(), key=lambda item: -item[1])]
    keys = keys[0:200]

    for w in keys:
      if w not in words:
        continue

      scores = {
        'ไม่ชอบหน้ากัน  (Dislike)': -2,
        'ไม่รู้จักกัน (Unfamiliar)': -1,
        'แค่คนรู้จักกัน (Acquainted)': 0,
        'สนิทกัน (Close)': 1,
        'สนิทกันมาก (Intimate)': 2,

        'ให้เกียรติมาก \n(Highly Respectful)': 2,
        'ให้เกียรติ (Respectful)': 1,
        'ปกติ \n(Normal)': 0,
        'ไม่ให้เกียรติ (Disrespectful)': -1,
        'ไม่ให้เกียรติมาก (Highly Disrespectful)': -2,
      }

      data.append({
          "word": w,
          "human": words[w],
          "human_score": scores[words[w]],
          "shapley": (lexicons[cat][w]),
          "category": cat,
          "path": path
      })

df = pd.DataFrame(data)

Loaded 2 records from Lexicons/v3/lexicon_task2_clse.jsonl
Loaded 2 records from Lexicons/v3/lexicon_task3_clse.jsonl


In [281]:
# df

In [283]:
o = [
    'ไม่ชอบหน้ากัน  (Dislike)',
    'ไม่รู้จักกัน (Unfamiliar)',
    'แค่คนรู้จักกัน (Acquainted)',
    'สนิทกัน (Close)',
    'สนิทกันมาก (Intimate)',
]

ho = [ '1. Close', '2. Know each other', "3. Don't know each other", "overall"]

from scipy import stats
for path in lexicon_paths:
  d1 = df[df["path"]==path]
  # print(path)
  # res = stats.spearmanr(d1["human_score"], d1["shapley"])
  # print(f"Overall Corr {res.statistic:.3f} {res.pvalue:.3f}")

  for c in ho:
    dd = d1[d1["category"]==c]
    # v = (dd.corr("spearman", numeric_only=True))
    # print(dd.groupby("human").count()["word"])
    res = stats.spearmanr(dd["human_score"], dd["shapley"])
    print(f"{c} {res.statistic:.3f} {res.pvalue:.3f}")
  print("=======")

1. Close 0.065 0.457
2. Know each other 0.222 0.013
3. Don't know each other 0.109 0.203
overall 0.158 0.057
1. Close 0.111 0.232
2. Know each other 0.044 0.665
3. Don't know each other 0.097 0.348
overall 0.053 0.575


In [278]:
o = [
    'ให้เกียรติมาก \n(Highly Respectful)',
    'ให้เกียรติ (Respectful)',
    'ปกติ \n(Normal)',
    'ไม่ให้เกียรติ (Disrespectful)',
    'ไม่ให้เกียรติมาก (Highly Disrespectful)',
]

ho = ['1. Respect', '2. Normal', '3. Not respect', "overall"]

from scipy import stats
for path in lexicon_paths:
  d1 = df[df["path"]==path]
  # print(path)
  # res = stats.spearmanr(d1["human_score"], d1["shapley"])
  # print(f"Overall Corr {res.statistic:.3f} {res.pvalue:.3f}")

  for c in ho:
    dd = d1[d1["category"]==c]
    # v = (dd.corr("spearman", numeric_only=True))
    # print(dd.groupby("human").count()["word"])
    res = stats.spearmanr(dd["human_score"], dd["shapley"])
    print(f"{c} {res.statistic:.3f} {res.pvalue:.3f}")
  print("=======")

1. Respect 0.078 0.288
2. Normal 0.141 0.053
3. Not respect -0.021 0.777
overall 0.165 0.025
1. Respect 0.091 0.207
2. Normal -0.057 0.431
3. Not respect 0.055 0.447
overall -0.054 0.460


In [285]:
# import matplotlib.pyplot as plt
# print(ho[0])
# # dd = df[df["category"]==ho[0]]
# # dd = df
# plot = sns.violinplot(data=dd, x="human", y="shapley", order=o, hue_order=ho)

In [272]:
# dd = df[df["category"]==ho[1]]
# plot = sns.violinplot(data=dd, x="human", y="shapley", order=o, hue_order=ho)

In [271]:
# dd = df[df["category"]==ho[2]]
# plot = sns.violinplot(data=dd, x="human", y="shapley", order=o, hue_order=ho)

In [ ]:
# from scipy import stats
# import pandas as pd

# def cal_corr(shap_lexicons, weights, label_fn):
#     df = []
#     for label in shap_lexicons:
#         count = defaultdict(int)
#         for w in shap_lexicons[label]:
#             if w not in lexicons:
#                 continue

#             for t in lexicons[w]:
#                 count[t] += 1

#         l = label_fn(label)
#         A = []
#         B = []
#         for feat in count:
#             k = (l, feat)
#             if k in weights and weights[k]!="":
#                 A.append(weights[k])
#                 B.append(count[feat])

#         spm = stats.spearmanr(A, B)
#         df.append({
#             "label": label,
#             "N": len(A),
#             "spearman": spm.statistic,
#             "pvalue": spm.pvalue
#         })

#     return pd.DataFrame(df)

In [ ]:
# shap_lexicons = load_jsonl("Classifier/lexicon_task1_clse.jsonl")[0]
# weights = load_obj_values("linear_weights_task1_clse.pkl")

# cal_corr(shap_lexicons, weights, closeness_to_eng1)

In [ ]:
# lexicon_paths = [
#   "Lexicons/v3/lexicon_task1_clse.jsonl",
#   "Lexicons/v3/lexicon_task2_clse.jsonl",
#   "Lexicons/v3/lexicon_task3_clse.jsonl",

#   "Lexicons/v3/lexicon_task1_auth.jsonl",
#   "Lexicons/v3/lexicon_task2_auth.jsonl",
#   "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# texts = []
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)
#   print(path)


#   N = 20
#   count = {}
#   # dict_keys(['pronoun', 'particles', 'misspelling'])
#   for cat in lexicons_by_feat:
#     for idx, w in enumerate(lexicons_by_feat[cat]["pronoun"]):
#       if idx >= N:
#         break

#       if w in count:
#         count[w].append(idx)
#       else:
#         count[w] = [idx]

#   words = {
#       (1, "same"):[],
#       (1, "div"):[],
#       (2, "same"):[],
#       (2, "div"):[],
#       (3, "same"):[],
#       (3, "div"):[],
#   }
#   for w in count:
#     n = len(count[w])

#     if n==1:
#       words[(n, "same")].append(w)
#     elif n>=2:
#       # print(w, count[w])
#       mx = max(count[w])
#       mn = min(count[w])

#       if mx-mn > 5:
#         words[(n, "div")].append((w, count[w]))
#       else:
#         words[(n, "same")].append((w, count[w]))

#   for n in words:
#     print(n, len(words[n]))
#     print(words[n])
#     print()
#   print("------------------")

In [69]:
# !wget -q http://www.arts.chula.ac.th/ling/wp-content/uploads/TH-Sarabun_Chula1.1.zip -O font.zip
# !unzip -qj font.zip TH-Sarabun_Chula1.1/THSarabunChula-Regular.ttf

import matplotlib
matplotlib.font_manager.fontManager.addfont('THSarabunChula-Regular.ttf')
matplotlib.rc('font', family='TH Sarabun Chula')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

def quadrant_chart(x, y, xtick_labels=None, ytick_labels=None, data_labels=None,
 highlight_quadrants=None, ax=None):
    """
    Create the classic four-quadrant chart.
    Args:
        x -- array-like, the x-coordinates to plot
        y -- array-like, the y-coordinates to plot
        xtick_labels -- list, default: None, a two-value list xtick labels
        ytick_labels -- list, default: None, a two-value list of ytick labels
        data_labels -- array-like, default: None, data point annotations
        highlight_quadrants -- list, default: None, list of quadrants to
            emphasize (quadrants are numbered 1-4)
        ax -- matplotlib.axes object, default: None, the user can pass their own
            axes object if desired
    """
    # allow user to specify their own axes
    ax = ax if ax else plt.axes()

    data = pd.DataFrame({'x': x, 'y': y, 'data_labels': data_labels})

    # calculate averages up front to avoid repeated calculations
    y_avg = data['y'].mean()
    x_avg = data['x'].mean()

    # set x limits
    adj_x = max((data['x'].max() - x_avg), (x_avg - data['x'].min())) * 1.1
    lb_x, ub_x = (x_avg - adj_x, x_avg + adj_x)
    ax.set_xlim(lb_x, ub_x)

    # set y limits
    adj_y = max((data['y'].max() - y_avg), (y_avg - data['y'].min())) * 1.1
    lb_y, ub_y = (y_avg - adj_y, y_avg + adj_y)
    ax.set_ylim(lb_y, ub_y)

    # set x tick labels
    if xtick_labels:
        ax.set_xticks([(x_avg - adj_x / 2), (x_avg + adj_x / 2)])
        ax.set_xticklabels(xtick_labels)

    # set y tick labels
    if ytick_labels:
        ax.set_yticks([(y_avg - adj_y / 2), (y_avg + adj_y / 2)])
        ax.set_yticklabels(ytick_labels, rotation='vertical', va='center')

    # determine which points to highlight
    if highlight_quadrants:
        quadrants = []
        for x_val, y_val in zip(x, y):
            q = []
            if (x_val >= x_avg) and (y_val >= y_avg):
                q.append(1)
            if (x_val <= x_avg) and (y_val >= y_avg):
                q.append(2)
            if (x_val <= x_avg) and (y_val <= y_avg):
                q.append(3)
            if (x_val >= x_avg) and (y_val <= y_avg):
                q.append(4)
            quadrants.append(q)
        data['quadrant'] = quadrants

        # boolean mask - True = highlight, False = don't highlight
        highlight = data['quadrant'].apply(lambda q: len(set(
        highlight_quadrants) & set(q)) > 0)

        # plot the non-highlighted points within the conditional block
        ax.scatter(data['x'][~highlight], data['y'][~highlight], alpha=0.5,
        c='lightblue', edgecolor='darkblue', zorder=99)
        data = data[highlight]

    # plot remaining points and quadrant lines
    ax.scatter(x=data['x'], y=data['y'], c='lightblue', edgecolor='darkblue',
    zorder=99)
    ax.axvline(x_avg, c='k', lw=1)
    ax.axhline(y_avg, c='k', lw=1)

    # add data labels
    for ix, row in data.iterrows():
        ax.annotate(row['data_labels'], (row['x'], row['y']), xytext=(2, 5),
        textcoords='offset pixels')

In [70]:
# lexicon_paths = [
#   "Lexicons/v3/lexicon_task1_clse.jsonl",
#   "Lexicons/v3/lexicon_task2_clse.jsonl",
#   "Lexicons/v3/lexicon_task3_clse.jsonl",

#   "Lexicons/v3/lexicon_task1_auth.jsonl",
#   "Lexicons/v3/lexicon_task2_auth.jsonl",
#   "Lexicons/v3/lexicon_task3_auth.jsonl"
# ]

# import numpy as np

# texts = []
# for path in lexicon_paths:
#   lexicons, lexicons_by_feat = load_jsonl(path)
#   print(path)


#   N = 50
#   count = {}
#   # dict_keys(['pronoun', 'particles', 'misspelling'])
#   for cat in lexicons_by_feat:
#     for idx, w in enumerate(lexicons_by_feat[cat]["misspelling"]):
#       if idx >= N:
#         break

#       if w in count:
#         count[w].append(idx)
#       else:
#         count[w] = [idx]

#   words = []
#   points = []

#   for w in count:
#     n = len(count[w])

#     if n==1:
#       x = count[w][0]
#       y = 0
#       continue
#     elif n>=2:
#       x = np.mean(count[w])
#       y = np.std(count[w])
#     points.append((x, y, w))



#   # plt.rcParams.update({'font.size': 25})
#   x = [p1 for p1, p2, _ in points]
#   y = [p2 for p1, p2, _ in points]
#   l = [p3 for p1, p2, p3 in points]

#   quadrant_chart(
#       x=x,
#       y=y,
#       xtick_labels=['Top', 'Bottom'],
#       ytick_labels=['Low', 'High'],
#       data_labels=l,
#       # highlight_quadrants=[2]
#   )
#   plt.title('', fontsize=16)
#   plt.ylabel('Variance', fontsize=14)
#   plt.xlabel('Average Rank', fontsize=14)
#   plt.show()
#   plt.clf()
#   # break